In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
from scipy import stats 
import matplotlib.pyplot as plt
from empiricaldist import Pmf , Cdf
from matplotlib.ticker import PercentFormatter

import warnings
warnings.filterwarnings("ignore")

In [2]:
fn_employment = pd.read_csv("fna02_employment_activity.csv")# fn = foreign nationals
fn_in_employment = pd.read_csv("fna08_foreign_nationals_in_employment.csv")

## EDA(Exploratory Data Analysis)

In [4]:
fn_in_employment.head()

,Statistic Label,Broad Nationality Group,Sex,Industry Sector NACE Rev 2,Year,UNIT,VALUE
0,Foreign Nationals in Employment,All nationalities,Both sexes,"Agriculture, forestry and fishing (A)",2005,Number,5116.0
1,Foreign Nationals in Employment,All nationalities,Both sexes,"Agriculture, forestry and fishing (A)",2006,Number,6032.0
2,Foreign Nationals in Employment,All nationalities,Both sexes,"Agriculture, forestry and fishing (A)",2007,Number,6705.0
3,Foreign Nationals in Employment,All nationalities,Both sexes,"Agriculture, forestry and fishing (A)",2008,Number,6863.0
4,Foreign Nationals in Employment,All nationalities,Both sexes,"Agriculture, forestry and fishing (A)",2009,Number,6154.0


In [42]:
print("CHECKING the shape: ",fn_in_employment.shape)#Chacking shape
print("\n CHECKING NULL values :\n",fn_in_employment.dtypes)#checking data types
print("\n Check info:\n",fn_in_employment.info)
print("\n COUNT is :\n",fn_in_employment.count() )#count rows
print("\n CHECKING NULL values :\n",fn_in_employment.isnull().sum())#checking null values


CHECKING the shape:  (6762, 7)

 CHECKING NULL values :
 Statistic Label                object
Broad Nationality Group        object
Sex                            object
Industry Sector NACE Rev 2     object
Year                            int64
UNIT                           object
VALUE                         float64
dtype: object

 Check info:
 <bound method DataFrame.info of                       Statistic Label Broad Nationality Group         Sex  \
0     Foreign Nationals in Employment       All nationalities  Both sexes   
1     Foreign Nationals in Employment       All nationalities  Both sexes   
2     Foreign Nationals in Employment       All nationalities  Both sexes   
3     Foreign Nationals in Employment       All nationalities  Both sexes   
4     Foreign Nationals in Employment       All nationalities  Both sexes   
...                               ...                     ...         ...   
6757  Foreign Nationals in Employment           American (US)      Female   


In [28]:
fn_employment.head()

,Statistic Label,Broad Nationality Group,Year of Entry,Year,UNIT,VALUE
0,Employment Activity of Foreign Nationals,All nationalities,All PPSN allocations to foreign nationals aged...,2002,Number,76919.0
1,Employment Activity of Foreign Nationals,All nationalities,All PPSN allocations to foreign nationals aged...,2003,Number,70138.0
2,Employment Activity of Foreign Nationals,All nationalities,All PPSN allocations to foreign nationals aged...,2004,Number,114750.0
3,Employment Activity of Foreign Nationals,All nationalities,All PPSN allocations to foreign nationals aged...,2005,Number,168197.0
4,Employment Activity of Foreign Nationals,All nationalities,All PPSN allocations to foreign nationals aged...,2006,Number,200450.0


In [46]:
fn_employment.describe()

,Year,VALUE
count,2499.00000,1370.000000
mean,2010.00000,10411.219708
std,4.89996,19218.666055
min,2002.00000,1.000000
25%,2006.00000,608.500000
50%,2010.00000,3551.000000
75%,2014.00000,10706.500000
max,2018.00000,200450.000000


In [39]:
print("CHECKING the shape: ",fn_employment.shape)#Chacking shape
print("\n CHECKING NULL values :\n",fn_employment.dtypes)#checking data types

print("\n COUNT is :\n",fn_employment.count())#count rows
print("\n CHECKING NULL values :\n",fn_employment.isnull().sum())#checking null values

CHECKING the shape:  (2499, 6)

 CHECKING NULL values :
 Statistic Label             object
Broad Nationality Group     object
Year of Entry               object
Year                         int64
UNIT                        object
VALUE                      float64
dtype: object

 COUNT is :
 Statistic Label            2499
Broad Nationality Group    2499
Year of Entry              2499
Year                       2499
UNIT                       2499
VALUE                      1370
dtype: int64

 CHECKING NULL values :
 Statistic Label               0
Broad Nationality Group       0
Year of Entry                 0
Year                          0
UNIT                          0
VALUE                      1129
dtype: int64


In [38]:
fn_in_employment.columns

Index(['Statistic Label', 'Broad Nationality Group', 'Sex',
       'Industry Sector NACE Rev 2', 'Year', 'UNIT', 'VALUE'],
      dtype='object')

## Splitting the dataset into the Training set and Test set

In [44]:
# Use only 'Age: 2' and 'EstimatedSalary: 3' features into X
#X = df.iloc[:, [2, 3]].values

# Store the 'Purchased' feature into 'y'
#y = df.iloc[:, -1].values
`
# Display X and y 
#print(X,y)

In [43]:
# Split the data set into Training and Testing parts
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Display the Number of rows and columns before (X, y) and after Splitting (X_train, X_test, y_train, y_test)
#X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape